In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications import EfficientNetB6, ResNet50, VGG16, Xception, VGG19, InceptionResNetV2, EfficientNetB3, EfficientNetB0, MobileNetV2
from keras.utils import plot_model

In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*6)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [12]:
base_dir = '../data/telemoji/pre'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

def make_blk(colar_pixel):
    test = np.mean(colar_pixel, axis=2, keepdims=True)
    test_array = np.concatenate([test,test,test], axis=2)
    return test_array

train_datagen = ImageDataGenerator(
    rescale=1/255,    
#     rotation_range=40,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
    validation_split=0.2,
    preprocessing_function = make_blk
)

test_datagen = ImageDataGenerator(
    rescale=1/255,
    preprocessing_function = make_blk
)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
#     color_mode='grayscale',
    target_size=(160,160),
    batch_size=100,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
#     color_mode='grayscale',
    target_size=(160,160),
    batch_size=100,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    validation_dir,
#     color_mode='grayscale',
    target_size=(160,160),
    batch_size=100,
    class_mode='categorical'
)

Found 15986 images belonging to 7 classes.
Found 3992 images belonging to 7 classes.
Found 2074 images belonging to 7 classes.


# 새로운 전이학습 모델

In [13]:
# pretrained network
model_base = MobileNetV2(include_top=False, 
                            weights='imagenet', input_shape=(160,160,3))
# model_base의 weight학습을 동결
model_base.trainable=False

model_base.summary()

model = Sequential()

# pretrained network를 우리의 모델 앞에 추가
model.add(model_base)

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.summary()

model.compile(optimizer=Adam(learning_rate=1e-4), 
              loss='categorical_crossentropy', metrics=['accuracy'])

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
flatten_4 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 32000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 224007    
Total params: 2,481,991
Trainable params: 224,007
Non-trainable params: 2,257,984
_________________________________________________________________


In [14]:
history = model.fit(train_generator, steps_per_epoch=159, epochs=20,
                    validation_data=validation_generator, validation_steps=39)

Epoch 1/20
159/159 [==============================] - 27s 168ms/step - loss: 2.5069 - accuracy: 0.3035 - val_loss: 1.7317 - val_accuracy: 0.4064
Epoch 2/20
159/159 [==============================] - 26s 161ms/step - loss: 1.9989 - accuracy: 0.3969 - val_loss: 1.6060 - val_accuracy: 0.4397
Epoch 3/20
159/159 [==============================] - 26s 162ms/step - loss: 1.7621 - accuracy: 0.4476 - val_loss: 1.5964 - val_accuracy: 0.4569
Epoch 4/20
159/159 [==============================] - 26s 161ms/step - loss: 1.5933 - accuracy: 0.4821 - val_loss: 1.5617 - val_accuracy: 0.4546
Epoch 5/20
159/159 [==============================] - 26s 162ms/step - loss: 1.4602 - accuracy: 0.5191 - val_loss: 1.5274 - val_accuracy: 0.4736
Epoch 6/20
159/159 [==============================] - 26s 162ms/step - loss: 1.3468 - accuracy: 0.5427 - val_loss: 1.5321 - val_accuracy: 0.4803
Epoch 7/20
159/159 [==============================] - 27s 169ms/step - loss: 1.2301 - accuracy: 0.5720 - val_loss: 1.5249 - val_ac

In [ ]:
result = model.evaluate(test_generator)
print(np.round(result, 4))

# Fine Tuning

In [ ]:
model_base.trainable=True

# 상위 layer 동결해제
for layer in model_base.layers:
    if layer.name in ['top_conv','block7b_project_conv', 'block7b_se_expand', 'block7b_se_reduce']:
        layer.trainable=True
    else:
        layer.trainable=False

# 미세조정이므로 learning_rate를 더 작게 설정
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=159, epochs=20,
                    validation_data=validation_generator, validation_steps=39)

In [ ]:
result = model.evaluate(test_generator)
print(result)